# Compare VGG16 Net Model
- Compare Keras vs onnx runtime

## VGG16 Net pretrained 모델 받기
- Keras로 vgg16 pretrained 모델 생성 
- imagenet 학습된 가중치 사용, test 는 일반 이미지 1장으로
- <b> .h5 모델 로드 후 .onnx로 변환 </b>


In [1]:
# load the model
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np

model = VGG16(weights='imagenet', include_top=True, input_shape = (224,224,3))
model.summary()

# 모델 저장
model.save('model/vggnet16.h5')
print('저장완료')

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Keras model to onnx
- .h5 to .onnx

In [5]:
# keras to onnx
import os
os.environ['TF_KERAS'] = '1'

import onnx
from keras2onnx import convert_keras
from tensorflow.keras.models import load_model
import onnxruntime as rt
import numpy as np

model = load_model('model/vggnet16.h5')
onx = convert_keras(model, 'vggnet16.onnx')

with open("model/vggnet16.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    
print('완료')

완료


## Image 전처리

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# load an image from file
image = load_img('mug.jpg', target_size=(224, 224))

# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# prepare the image for the VGG model
image = preprocess_input(image)

## Inference time 비교

In [12]:
import time

# Keras
# 시간측정
start = time.time()
for i in range(10):
    yhat = model.predict(image)
end = time.time()
print('keras - vggnet16 걸린시간:', np.double(end-start))

# ONNX
# Compute the prediction with ONNX Runtime
sess = rt.InferenceSession('model/vggnet16.onnx')
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# 시간측정
start = time.time()
for i in range(10):
    pred_onx = sess.run([label_name], {input_name: image.astype(np.float32)})[0]
end = time.time()

print('onnxruntime - vggnet16 걸린시간:', np.double(end-start))

keras - vggnet16 걸린시간: 3.5737996101379395
onnxruntime - vggnet16 걸린시간: 3.503628730773926
